In [6]:
import gym
import random
import numpy as np
from statistics import mean, median
from collections import Counter
import tensorflow as tf
tf.disable_eager_execution()
import matplotlib.pyplot as plt
%matplotlib

Using matplotlib backend: Qt5Agg


In [7]:
LR = 1e-3
env = gym.make('CartPole-v1')
env.reset()
TRIALS = 20
params = dict(
    goal_steps = 500,
    score_requirement = 30,
    initial_games = 500,
    X = np.array([[]]),
    Y = np.array([]),
    action = 2,
    score = np.array([0]),
    plot = False,
    model = None
)

CartPoleEnv - Version 0.2.0, Noise case: 1


In [8]:
def plot(scores=[]):
    plt.clf()
    avg_scores = [sum(scores[:index+1])/(index+1) for index, 
                  score in enumerate(scores)]
    avg_score = sum(scores)/len(scores)
    x_val = [0,len(scores)-1]
    y_val = [avg_score, avg_score]
    plt.plot(scores,'g-', label='current score')
    plt.plot(x_val, y_val,'r-', label='average score')
    plt.plot(avg_scores,'b-', label='average scores')
    plt.xlabel('Episodes--->')
    plt.ylabel('Score--->')
    plt.legend()
    plt.plot()

In [9]:
def data(params):
    env.reset()
    for game in range(params["initial_games"]):
        env.reset()
        game_data = np.array([])
        prev_observation = np.array([])
        action_list = np.array([])
        for step in range(params["goal_steps"]):
            if not prev_observation.size or params['model'] is None:
                action = env.action_space.sample()
            else :
                prob_values = model.predict(np.array([prev_observation]))[0]
                action = np.argmax(prob_values)
            observation, reward, done, info =  env.step(action)
            params['score'][-1] += reward
            if prev_observation.size :
                if not game_data.size:
                    game_data = prev_observation
                    action_list = np.array([action])
                else:
                    game_data = np.vstack((game_data, prev_observation))
                    action_list = np.concatenate((action_list,action), axis = None)
            if done:
                break
            prev_observation = observation
        if params['plot']:
            plot(params['score'])
        if params['score'][-1] >= params['score_requirement']:
            if not params['X'].size:
                params['X'] = game_data
                params['Y'] = action_list
            else:
                params['X']=np.concatenate((params['X'], game_data), axis=0)
                params['Y']=np.concatenate((params['Y'], action_list))
        params['score'] = np.concatenate((params['score'], 0), axis = None)
data(params)

In [10]:
model = tf.keras.Sequential([tf.keras.layers.Dense(64,input_shape=(4,),activation='relu'),
                             tf.keras.layers.Dense(128,activation='relu'),
                             tf.keras.layers.Dense(2, activation='softmax'),

])
final_score_list = params['score']
for trial in range(TRIALS):    
    model.compile(loss='sparse_categorical_crossentropy', optimizer='adam')
    if params['X'].size > 0:
        model.fit(params['X'],params['Y'], epochs=10)
    else:
        break
    params = dict(
        goal_steps = 500,
        score_requirement = 50 + trial * 20,
        initial_games = 150,
        X = np.array([[]]),
        Y = np.array([]),
        action = 2,
        score = np.array([0]),
        plot = False,
        model = model
    )
    data(params)
    final_score_list = np.concatenate((final_score_list, params['score']), axis=None)
plot(final_score_list)

Train on 686 samples
Epoch 1/10
686/686 [==============================] - 0s 674us/sample - loss: 0.6921
Epoch 2/10
686/686 [==============================] - 0s 56us/sample - loss: 0.6820
Epoch 3/10
686/686 [==============================] - 0s 71us/sample - loss: 0.6743
Epoch 4/10
686/686 [==============================] - 0s 61us/sample - loss: 0.6724
Epoch 5/10
686/686 [==============================] - 0s 74us/sample - loss: 0.6678
Epoch 6/10
686/686 [==============================] - 0s 73us/sample - loss: 0.6678
Epoch 7/10
686/686 [==============================] - 0s 55us/sample - loss: 0.6626
Epoch 8/10
686/686 [==============================] - 0s 70us/sample - loss: 0.6616
Epoch 9/10
686/686 [==============================] - 0s 62us/sample - loss: 0.6585
Epoch 10/10
686/686 [==============================] - 0s 52us/sample - loss: 0.6593
Train on 7626 samples
Epoch 1/10
7626/7626 [==============================] - 1s 106us/sample - loss: 0.3013
Epoch 2/10
7626/7626 [======

In [ ]:
model.save("DL_model_task2.h5")

In [8]:
params = dict(
    goal_steps = 500,
    score_requirement = 0,
    initial_games = 100,
    X = np.array([[]]),
    Y = np.array([]),
    action = 2,
    score = np.array([0]),
    plot = True,
    model = model
)
data(params)